In [ ]:
# # Install TF-Hub.
# !pip install tensorflow-hub
# !pip install seaborn

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import pdb
import shutil
from nltk.tokenize import word_tokenize
from typing import Tuple
from sklearn.model_selection import ParameterGrid
from collections import OrderedDict
import json
from pprint import pprint
# import logging
# logging.basicConfig(format='%(levelname)s %(asctime)s : %(message)s', level=logging.INFO)

In [ ]:
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
!export TFHUB_CACHE_DIR=~/tf_hub_cache

In [ ]:
HUB_LINK_DICT = {
    'use_dan': "https://tfhub.dev/google/universal-sentence-encoder/2",
    'use_large': "https://tfhub.dev/google/universal-sentence-encoder-large/3",
    'nnlm': "https://tfhub.dev/google/nnlm-en-dim128/1",
    'elmo': "https://tfhub.dev/google/elmo/2",
    'w2v': "https://tfhub.dev/google/Wiki-words-500-with-normalization/1",
}

In [ ]:
# Load all files from a directory in a DataFrame.
def load_yelp_train_test(path='pytorch-pretrained-BERT/examples/data'):
    train_df = pd.read_csv(os.path.join(path, 'yelp_class_train.csv'), encoding='latin1')
    test_df = pd.read_csv(os.path.join(path, 'yelp_class_test.csv'), encoding='latin1')
    return train_df, test_df

def make_model_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
    else:
        shutil.rmtree(path)
        os.makedirs(path)

In [ ]:
class TFHubCompositionalitySwitchingPipeline(object):
    def __init__(self, model_name: str, finetune_module: bool, last_layer: str = 'linear', num_classes: int = 2,
                 lr: float = 0.003, eval_on_train: bool = True, continue_from_ckpt: bool = False, **kwargs):
        self.model_name = model_name
        self.finetune_module = finetune_module
        self.last_layer = last_layer
        self.model_id = self.model_name + '_' + str(self.finetune_module) + '_' + self.last_layer
        print("Model config = {}".format(self.model_id))
        self.num_classes = num_classes
        self.lr = lr
        self.eval_on_train = eval_on_train
        self.continue_from_ckpt = continue_from_ckpt

        self.model_url = HUB_LINK_DICT[self.model_name.lower()]
        self.train_df, self.test_df = self.load_data()

        self.model_config = None
        self.estimator = None
        self.data_dir = 'pytorch-pretrained-BERT/examples/data/'
        self.eval_dict = dict()
        self.eval_dict['model_id'] = self.model_id
        print(self.train_df.head())
        print(self.train_df.shape)
        print(self.test_df.shape)

    def load_data(self) -> Tuple[pd.DataFrame, pd.DataFrame]:
        return load_yelp_train_test()

    def build_model(self) -> None:
        if not self.continue_from_ckpt:
            print("Clearing old checkpoints and making new model dir")
            make_model_dir(self.model_id)
        embedded_text_feature_column = hub.text_embedding_column(
            key="text",
            module_spec=self.model_url,
            trainable=self.finetune_module)
        self.model_config = tf.estimator.RunConfig(
            model_dir=self.model_id,
            tf_random_seed=123,
            save_summary_steps=1000,
            keep_checkpoint_max=2,
            log_step_count_steps=1000,
        )
        if self.last_layer == 'linear':
            self.estimator = tf.estimator.LinearClassifier(
                feature_columns=[embedded_text_feature_column],
                n_classes=self.num_classes,
                optimizer=tf.train.AdagradOptimizer(learning_rate=self.lr),
                config=self.model_config)
        elif self.last_layer == 'dnn':
            self.estimator = tf.estimator.DNNClassifier(
                hidden_units=[500, 100],
                feature_columns=[embedded_text_feature_column],
                n_classes=self.num_classes,
                optimizer=tf.train.AdagradOptimizer(learning_rate=self.lr),
                config=self.model_config)
        else:
            raise ValueError("Invalid last layer value, valid options: linear or dnn")

    def train_model(self, num_epochs: int = 3) -> None:
        # Training input on the whole training set with no limit on training epochs.
        train_input_fn = tf.estimator.inputs.pandas_input_fn(
            self.train_df, self.train_df["label"], num_epochs=num_epochs, shuffle=True)

        self.estimator.train(input_fn=train_input_fn)

    def eval_model(self) -> None:
        if self.eval_on_train:
            # Prediction on the whole training set.
            predict_train_input_fn = tf.estimator.inputs.pandas_input_fn(
                self.train_df, self.train_df["label"], shuffle=False)
            train_eval_result = self.estimator.evaluate(input_fn=predict_train_input_fn)
            print("Training set accuracy: {accuracy}".format(**train_eval_result))
            self.eval_dict['train_accuracy'] = train_eval_result['accuracy']

        # Prediction on the test set.
        predict_test_input_fn = tf.estimator.inputs.pandas_input_fn(
            self.test_df, self.test_df["label"], shuffle=False)
        test_eval_result = self.estimator.evaluate(input_fn=predict_test_input_fn)
        print("Test set accuracy: {accuracy}".format(**test_eval_result))
        self.eval_dict['test_accuracy'] = test_eval_result['accuracy']

    def compositionality_eval(self) -> None:
        if self.estimator is None:
            raise ValueError("Build self.estimator by calling self.build_model() before evaluation")
        comp_df = pd.read_csv(os.path.join(self.data_dir, 'compositionality_testing.csv'), encoding='latin1')
        comp_df['text'] = comp_df['text1']
        predict_test_input_fn1 = tf.estimator.inputs.pandas_input_fn(comp_df, comp_df["label1"], shuffle=False)

        comp_df['text'] = comp_df['text0']
        predict_test_input_fn0 = tf.estimator.inputs.pandas_input_fn(comp_df, comp_df["label0"], shuffle=False)

        comp_df['preds0'] = [int(x["class_ids"][0]) for x in self.estimator.predict(input_fn=predict_test_input_fn0)]
        comp_df['preds1'] = [int(x["class_ids"][0]) for x in self.estimator.predict(input_fn=predict_test_input_fn1)]
        comp_df['correct'] = comp_df.apply(lambda x: int(x['preds0'] == 0 and x['preds1'] == 1), axis=1)
        self.eval_dict['comp_accuracy_all'] = comp_df['correct'].mean()

        comp_df['words_added'] = comp_df.apply(
            lambda x: (set(word_tokenize(x['text1'])) - set(word_tokenize(x['text0']))), axis=1)
        comp_df['num_words_added'] = comp_df.apply(lambda x: len(x['words_added']), axis=1)
        comp_df = comp_df[comp_df['num_words_added'] > 0].reset_index(drop=True)
        self.eval_dict['comp_accuracy_filter'] = comp_df['correct'].mean()
        print("Switching accuracy Filtered= {}".format(comp_df['correct'].mean()))
        save_file = os.path.join(self.data_dir, 'compositionality_pred_{}.csv'.format(self.model_id))
        comp_df.to_csv(save_file, index=False)

    def run_all(self) -> None:
        self.build_model()
        self.train_model()
        self.eval_model()
        self.compositionality_eval()


In [ ]:
# pipeline = TFHubCompositionalitySwitchingPipeline('use_dan', True, 'linear')
# pipeline.run_all()
# pipeline.eval_dict

In [ ]:
param_vals = {'model_name': ['use_dan', 'use_large'], 'finetune_module': [True, False], 'last_layer': ['dnn']}
param_grid = list(ParameterGrid(param_vals))
pprint(param_grid)
model_log_dict = OrderedDict()

In [ ]:
for params in param_grid:
    pipeline = TFHubCompositionalitySwitchingPipeline(**params)
    pipeline.run_all()
    model_log_dict[pipeline.model_id] = pipeline.eval_dict

In [ ]:
with open('model_log_dict_dnn.json', 'w') as f:
    json.dump(str(model_log_dict), f, indent=4)

In [ ]:
model_log_df = pd.DataFrame.from_dict(model_log_dict, orient='index')
model_log_df.to_csv('model_log_df_use_dnn.csv', index=True)

In [ ]:
model_log_df